# Quantum Chemistry Calculations

This Jupyter notebook contains several examples to illustrate use of the Psi-4 quantum chemistry package in running ab initio calculations.These examples can be used to complete the workshops; We will use ASE to build and view structures

In all of the following examples, a single water molecule is chosen as a convenient example molecular system. Of course, these code snippets could be used to perform calculations for other molecular systems by simply changing the coordinates.

Examples included here are: 

1. Hartree-Fock energy calculation;
3. Hartree-Fock geometry optimization; 
4. Outputting useful information; 
5. Normal-mode analysis using HF;
6. Transition-state optimization;
7. Examples of visualization using ASE;
8. Alternative geometry definitions.
9. Hartree-Fock geometry optimization for open-shell system;
10. Hartree-Fock geometry optimization for charged species.

## Documentation

Useful documentation for *psi4* can be found here:

https://psicode.org/psi4manual/master/psiapi.html

I note that we are using *psi4* as a *Python* module, rather than as an executable - the link above goes to the relevant documentation.

Documentation for energy calculationss, optimization calculations and frequency calculations are here:

https://psicode.org/psi4manual/master/api/psi4.driver.energy.html#psi4.driver.energy

https://psicode.org/psi4manual/master/api/psi4.driver.optimize.html#psi4.driver.optimize

https://psicode.org/psi4manual/master/api/psi4.driver.frequency.html#psi4.driver.frequency

Finally, if the above documentation or the examples below don't help - try google!

## Trouble-shooting

Sometimes, you'll find that your *psi4* calculations fail, often with error messages which can be difficult to interpret! Here are a few hints and tips I have based on my own attempts:

- If you find that a geometry optimization calculation fails to converge, try adding the following line to your python code, just before you run *psi4.optimize()*:

        psi4.set_options({'opt_coordinates': 'both'})

    This changes the coordinate system that *psi4* uses for optimization - usually this helps!


- Another tricky issue relates to symmetry - if you find that calculations fail to converge and the error message is mentioning symmetry, you could try adding:
        
        symmetry c1

    to the last line of your *psi4.geometry* block [i.e. on the line before the final **""")**]
    

- Sometimes, no matter what you do, you might find that your *psi4* calculations keep on failing, even though there doesn't seem to be anything actually wrong with the input. In these cases, click on the *Kernel* menu above and then click on *Restart & clear output*. After doing this, you'll have to run the notebook set-up cells again because the *psi4* module won't be loaded any more!

## Calculation set-up

This is important - before we run any calculations, we need to import the packages that we're going to use; if we skip this step, nothing will work!

Note that you need to make sure that these packages are loaded before you run any calculations - it's best practice to place these packages somewhere near the top of your workbook, and to re-run these cells whenever you re-open your workbook.

In [2]:
# Import the psi4 package - required to run the calculations!
import psi4
psi4.set_memory('1 GB');

In [3]:
# Import ASE for viewing....
import ase
from ase import io
from ase.visualize import view

In [4]:
# Import numpy - not explicitly used here, but just a placeholder.
import numpy as np

# Import matplotlib.
from pylab import *
import matplotlib.pyplot as plt

## Example 1: Hartree-Fock energy calculation

This is a standard Hartree-Fock calculation for a single water molecule, with the geometry defined using the Cartesian (x,y,z) coordinates of each atom (in Angstroms).

In [5]:
# Sample HF calculation with a cc-pVDZ basis set. 
#

h2o = psi4.geometry("""
O       -0.9228114122      0.9383318842      0.0000000000                 
H        0.0471885878      0.9383318842      0.0000000000                 
H       -1.2461412239      0.0789766302     -0.3128360279 
""")

E = psi4.energy('hf/cc-pvdz')
print('The HF energy is ',E,' Hartrees ')

The HF energy is  -76.02501650035721  Hartrees 


## Example 2: HF geometry optimization

Here, we perform a geometry optimization calculation using Hartree-Fock and a cc-pVDZ basis set.

In [6]:
# Sample HF geometry optimization. 

h2o = psi4.geometry("""
O       -0.9228114122      0.9383318842      0.0000000000                 
H        0.0471885878      0.9383318842      0.0000000000                 
H       -1.2461412239      0.0789766302     -0.3128360279 
""")
psi4.optimize('hf/cc-pvdz')

Optimizer: Optimization complete!


-76.02703278102143

In [7]:
# Note that, if we have a previously defined molecule (defined as a psi4.geometry)
# we can also invoke the geometry optimization using....

psi4.optimize('hf/cc-pvdz',mol=h2o)

Optimizer: Optimization complete!


-76.02703278102084

## Example 3: Getting useful output

In the examples above, we just see the final energy values being printed out - it's obviously more useful to have access to more of the information that's created during a calculation. Here, we'll look at some examples.

In [8]:
# We can set an output filename where useful information is stored using the following:

psi4.core.set_output_file('output.dat')

# Note that 'output.dat' can be any string you'd like - for example, you can have different output 
# files for different calculations.


In [9]:
# Let's now run the Hartree-Fock geometry optimization again....you should get a file
# called output.dat created in this same directory where you're running the calculation.
psi4.optimize('hf/cc-pvdz',mol=h2o)

Optimizer: Optimization complete!


-76.02703278102088

In [11]:
# This example shows how we can print out the atomic coordinates in Angstroms.

# First, let's reset the input H2O molecule geometry to the original geometry.

psi4.core.set_output_file('geometry_optimization.dat')

h2o = psi4.geometry("""
O       -0.9228114122      0.9383318842      0.0000000000                 
H        0.0471885878      0.9383318842      0.0000000000                 
H       -1.2461412239      0.0789766302     -0.3128360279 
""")

# Let's print this out to the screen....
print("* Geometry BEFORE optimization...")
print( h2o.save_string_xyz() )

psi4.optimize('hf/cc-pvdz', molecule = h2o)

print("\n* Geometry AFTER optimization...")
print( h2o.save_string_xyz() )


* Geometry BEFORE optimization...
0 1
 O   -0.000000000000    0.000000000000    0.062675830414
 H    0.792000605234    0.000000000000   -0.497355455624
 H   -0.792000605234   -0.000000000000   -0.497355455624

Optimizer: Optimization complete!

* Geometry AFTER optimization...
0 1
 O    0.000000000000   -0.000000000000    0.064748966298
 H    0.748773780078    0.000000000000   -0.513806541077
 H   -0.748773780078   -0.000000000000   -0.513806541077



You can see the page here: http://www.psicode.org/psi4manual/master/api/psi4.core.Molecule.html
to see more routines which can be used to inquire about molecules. 

In [12]:
# Further examples - note that these outputs will be printed to 'output_test.dat'.

psi4.core.set_output_file('output_test.dat')

# Calculates the NxN distance matrix in Bohr - i.e. outputs distance between all pairs of atoms.
M = h2o.distance_matrix();
print( M.print_out() );

# Output atomic coordinates of a specific atom (reminder: numbering beings at zero in Python)
print('* XYZ coordinates of second atom are: ', h2o.xyz(1))


None
* XYZ coordinates of second atom are:  [ 1.41498, 4.33212e-17, -0.970954 ]


## Example 4: Frequency calculation

In this example, we'll perform a normal-mode analysis for our H2O molecule, giving the vibrational frequencies. 

In [13]:
# Set the output file for this calculation.
psi4.core.set_output_file('frequencies.dat')

# Set the geometry....
h2o = psi4.geometry("""
O       -0.9228114122      0.9383318842      0.0000000000                 
H        0.0471885878      0.9383318842      0.0000000000                 
H       -1.2461412239      0.0789766302     -0.3128360279 
""")

# Optimize the geometry using HF/cc-pVDZ.
psi4.optimize('hf/cc-pvdz', molecule = h2o)

# Calculate the frequencies - note that this is performed for the optimized geometry given
# by the above routine.
psi4.frequencies('hf/cc-pvdz', molecule = h2o)

Optimizer: Optimization complete!


-76.02703278102084

After running the above, have a look in 'frequencies.dat'. If you scroll all the way to the bottom, you should be able to find the three harmonic vibrational frequencies for H2O.

You can also find the thermochemistry calculation at the bottom of the 'frequencies.dat'!

## Example 5: Transition-state optimization

In this example, we'll look at the transition-state optimization for the reaction of NO2 with ethene. The initial geometry (tsmol) is a good approximation of the transition-state; output is sent to TSopt.dat.

Note that the TSopt.dat file contains the output and, at the bottom, contains details on the thermochemistry and frequency calculation. You should check that one of the vibrational frequencies is imaginary (should be about 2183i cm-1) - this is an indicator that there was a single negative frequency (eigenvalue) in the Hessian.


In [14]:
psi4.core.set_output_file('TSopt.dat')
tsmol = psi4.geometry("""
  C   0.00288769176631      0.14234132324571     -0.00000000534925
  O   1.14977129175014     -0.01062150758604      0.00000000212064
  H   -0.89961837469095     -1.33546412611440      0.00000084033018
  H   -1.07846840882550     -0.03660121954528      0.00000027289843
""")

# TS optimization
psi4.set_options({'opt_type': 'ts'})
psi4.optimize('hf/6-31g')

# Frequency calculation to confirm imaginary frequency in Hessian.

psi4.frequencies('hf/6-31g')

Optimizer: Optimization complete!


-113.62977752436898

## Example 6: Visualization using ASE

In this example, we'll simply render the tsmol molecule generated by the TS search above. We do this in three steps:
- First, output the tsmol object to an xyz file.
- Second, we read in the ts.xyz file into an ASE object.
- Third, we use ASE's view facility.



In [15]:
# First save the molecule tsmol to a file called ts.xyz...

tsmol.save_xyz_file('ts.xyz',True)

# Next load ts.xyz as an ASE atoms object...

m = io.read('ts.xyz')

# ...and now view it! (Note that this all relies on loading ASE 
# right at the top of the workbook).

view(m,viewer='x3d')

# You should now see a figure below showing the transition-state geometry.


## Example 7: Alternative molecular geometry definition

In all of the examples above, we've defined our input molecular geometry using atomic Cartesian coordinates. That is, we've given the (x,y,z) coordinate of each atom in the structure.

An alternative structure definition which can be used is the Z-matrix (https://en.wikipedia.org/wiki/Z-matrix_(chemistry). Here, the molecule is defined using a set of interatomic distances, bond angles and torsion angles. 

The geometric information contained in the Cartesian and Z-matrix representations - however, Z-matrix representation is often more useful, for example when scanning over internal coordinates such as a torsion angle. 

Below is an example calculation where the geometry is defined as a Z-matrix.


In [16]:
# Define a new water molecule using a Z-matrix. The two OH bond lengths are set here 
# as 0.97 Angstroms, and the bond angle is 104.5 degrees....

h2o_Zmatrix = psi4.geometry("""
   O
   H 1 0.97
   H 1 0.97 2 105.0
""")
E = psi4.energy('hf/cc-pvdz', molecule = h2o_Zmatrix)
print('The HF energy is ',E,' Hartrees ')


The HF energy is  -76.02583436101241  Hartrees 


In [17]:
# Another example - this is HOOH, with a torsion angle of 0 degrees (so the two hydrogen
# atoms are eclipsed).
#
hooh = psi4.geometry("""
   symmetry c1
   H
   O 1 0.946347
   O 2 1.397780 1  107.243777
   H 3 0.946347 2  107.243777   1 0.0
""")

E = psi4.energy('hf/cc-pvdz', molecule = hooh)
print('The HF energy is ',E,' Hartrees ')


The HF energy is  -150.7738832670609  Hartrees 


In [18]:
# Let's have a look at the molecule....

hooh.save_xyz_file('hooh_1.xyz',True)
m = io.read('hooh_1.xyz')
view(m,viewer='x3d')

In [19]:
# Now let's change the torsion angle to 180 degrees....
hooh = psi4.geometry("""
   symmetry c1
   H
   O 1 0.946347
   O 2 1.397780 1  107.243777
   H 3 0.946347 2  107.243777   1 180.0
""")

E = psi4.energy('hf/cc-pvdz', molecule = hooh)
print('The HF energy is ',E,' Hartrees ')

The HF energy is  -150.78153749772764  Hartrees 


In [20]:
# Let's have a look at the molecule....
hooh.save_xyz_file('hooh_2.xyz',True)
m = io.read('hooh_2.xyz')
view(m,viewer='x3d')

## Example 8: HF geometry optimization for open-shell species

Here, we perform a geometry optimization calculation using Hartree-Fock and a cc-pVDZ basis set. However, in this case, the molecule is not a closed-shell species - there is one unpaired electron. 

To account for this, we add an extra line to the geometry definition. Here, you can see that the first line in the geometry is now "0 2" - here, the "0" defines the total charge, and "2" is the spin-multiplicity, given by 2S+1, where S is the total spin angular momentum. In the case of a single unpaired electron (as in this radical species example), S=1/2 so we have 2S+1 = 2.

Finally, also notice that we've added the line:

    psi4.set_options({'reference': 'uhf'})
    
before we run the optimization. This makes sure that the Hartree-Fock calculation is set up in an "unrestricted" calculation; we don't need to be too concerned with the details, but this is important when accounting for open-shell species in *ab initio* calculations using Hartree-Fock (If you're interested in the details, google "Unrestricted Hartree Fock" for further information).

In [23]:
# C2H geometry optimization.

c2h = psi4.geometry("""
0 2
C       0.0000000      0.0000000     -0.7388934234                 
C       0.0000000     0.0000000      0.4870691299                 
H       0.0000000      0.0000000      1.5706017606 
symmetry c1
""")
psi4.set_options({'reference': 'uhf'})
psi4.set_options({'opt_coordinates': 'both'})
E = psi4.optimize('hf/6-31g',molecule=c2h)
print("Reactant energy = ",E)


Optimizer: Optimization complete!
Reactant energy =  -76.12882039150372


## Example 9: HF geometry optimization for a charged system

Here, we perform a geometry optimization calculation using Hartree-Fock and a cc-pVDZ basis set. However, in this case, the molecule is charged.

As in the example above, we define the charge and spin-multiplicity in the first line as "-1 1". Here, the charge is "-1", but there are no unpaired electrons, so S=0 and 2S+1 = 1. As a result, the "*{charge} {multiplicity}* line says "-1 1".


In [24]:
c2h = psi4.geometry("""
-1 1
O       0.00000  0.00000  0.00000                  
H       1.0      0.000000   0.0000000000                  
""")
#psi4.set_options({'reference': 'uhf'})
E = psi4.optimize('hf/6-31g',molecule=c2h)
print("Reactant energy = ",E)

Optimizer: Optimization complete!
Reactant energy =  -75.31173172125216
